# InSightAI: Multi-Agent Data Analysis Demo

[<img src="https://colab.research.google.com/assets/colab-badge.svg" align="left">](https://colab.research.google.com/github/LeoRigasaki/InSightAI/blob/main/demo.ipynb)

<br><br>

Welcome to the official InSightAI demonstration notebook. This notebook showcases how InSightAI leverages multiple AI agents to perform complex data analysis, SQL querying, and automated reporting.

### 🗝️ Prerequisites
Ensure you have your API keys set in a `.env` file or environment variables:
- `GROQ_API_KEY`, `GEMINI_API_KEY`, or `OPENAI_API_KEY`.

In [3]:
import os
import pandas as pd
import sqlite3
from dotenv import load_dotenv
from insightai import InsightAI

# Load environment variables
load_dotenv()

# Initialize InsightAI
ai = InsightAI()
print("✅ InSightAI Initialized")

✅ InSightAI Initialized


## 📊 Mode 1: Interactive CSV Analysis

In this mode, InSightAI analyzes a CSV file by dynamically selecting the best 'Expert' for the task and generating/executing Python code.

In [4]:
# Load sample sales data
df = pd.read_csv('sales_data.csv')
ai.df = df

# Ask a complex question
ai.pd_agent_converse("Analyze the monthly revenue trend and identify if we have any seasonality.")

```json
{
  "requires_dataset": true,
  "expert": "Data Analyst",
  "confidence": 9
}
```


```json
{
  "analyst": "Data Analyst DF",
  "unknown": "Monthly revenue trend and whether seasonality exists",
  "data": "Pandas DataFrame 'df' with columns ['Order ID', 'Order Date', 'Customer ID', 'Product Category', 'Region', 'Sales', 'Profit', 'Discount', 'Ship Mode']",
  "condition": "Aggregate monthly revenue from 'Sales' using 'Order Date', analyze the trend, and determine if seasonality is present"
}
```


Reflection on the problem  
We need to detect seasonality in monthly revenue, but the supplied data only spans 28 calendar days (2024-11-08 → 2024-12-05).  With only one partial month (November) and a few December days we do not have enough full months to observe repeating yearly patterns.  The only numeric revenue-like column is “Sales”; there is no explicit “Revenue” column, so we will treat Sales as the revenue proxy.  The task is therefore reduced to:  
1. Convert the string “Order Date” to a true datetime.  
2. Aggregate daily Sales to daily revenue.  
3. Aggregate daily revenue to calendar-month revenue (November 2024 and December 2024).  
4. Inspect the two monthly totals and explicitly state that with only two partial months no statistical seasonality can be inferred.  
5. (Optional) show the daily trend within the available window for completeness.

```yaml
plan:
  - "Step 1: Convert the 'Order Date' column from string to pandas datetime."
  - "Step 2: Create a new column 'Rev

## Reasoning:

Reflection on the problem  
We need to detect seasonality in monthly revenue, but the supplied data only spans 28 calendar days (2024-11-08 → 2024-12-05).  With only one partial month (November) and a few December days we do not have enough full months to observe repeating yearly patterns.  The only numeric revenue-like column is “Sales”; there is no explicit “Revenue” column, so we will treat Sales as the revenue proxy.  The task is therefore reduced to:  
1. Convert the string “Order Date” to a true datetime.  
2. Aggregate daily Sales to daily revenue.  
3. Aggregate daily revenue to calendar-month revenue (November 2024 and December 2024).  
4. Inspect the two monthly totals and explicitly state that with only two partial months no statistical seasonality can be inferred.  
5. (Optional) show the daily trend within the available window for completeness.

```yaml
plan:
  - "Step 1: Convert the 'Order Date' column from string to pandas datetime."
  - "Step 2: Create a new column 'Revenue' equal to the 'Sales' column (since Sales is the only monetary measure)."
  - "Step 3: Group by the calendar month (year-month) and sum the Revenue to obtain monthly revenue totals."
  - "Step 4: Count how many complete calendar months are present; if fewer than 24, conclude that seasonality cannot be detected."
  - "Step 5: Output the monthly totals for the available months (2024-11 and 2024-12) and explicitly state the limitation."
  - "Step 6: (Optional) Plot daily revenue within the 28-day window to visualise the short-term trend."
```

```python
# -----------------------------------------------------------
# Analyze monthly revenue trend and identify seasonality
# -----------------------------------------------------------

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Ensure the folder for plots exists
os.makedirs('visualization', exist_ok=True)

# --- 1. Make sure we have the dataframe -------------------------------------------------
# df is assumed to be already defined

# --- 2. Convert Order Date to datetime -------------------------------------------------
try:
    df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')
except Exception as e:
    print("Error converting Order Date:", e)

# --- 3. Create Revenue column ----------------------------------------------------------
df['Revenue'] = df['Sales']  # Sales is the monetary measure

# --- 4. Aggregate monthly revenue ----------------------------------------------------
df['YearMonth'] = df['Order Date']

## Dataframe Preview

,Order ID,Order Date,Customer ID,Product Category,Region,Sales,Profit,Discount,Ship Mode,Revenue,YearMonth
0,CA-2024-152156,2024-11-08,CG-12520,Furniture,South,261.96,41.91,0.00,Second Class,261.96,2024-11
1,CA-2024-152156,2024-11-08,CG-12520,Furniture,South,731.94,219.58,0.00,Second Class,731.94,2024-11
2,CA-2024-138688,2024-06-12,DV-13045,Office Supplies,West,14.62,6.87,0.00,Second Class,14.62,2024-06
3,US-2024-108966,2024-10-11,SO-20335,Furniture,Central,957.57,-383.03,0.45,Standard Class,957.57,2024-10
4,US-2024-108966,2024-10-11,SO-20335,Office Supplies,Central,22.37,2.52,0.20,Standard Class,22.37,2024-10


## Applied Code:

```python
# -----------------------------------------------------------
# Analyze monthly revenue trend and identify seasonality
# -----------------------------------------------------------

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Ensure the folder for plots exists
os.makedirs('visualization', exist_ok=True)

# --- 1. Make sure we have the dataframe -------------------------------------------------
# df is assumed to be already defined

# --- 2. Convert Order Date to datetime -------------------------------------------------
try:
    df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')
except Exception as e:
    print("Error converting Order Date:", e)

# --- 3. Create Revenue column ----------------------------------------------------------
df['Revenue'] = df['Sales']  # Sales is the monetary measure

# --- 4. Aggregate monthly revenue ----------------------------------------------------
df['YearMonth'] = df['Order Date'].dt.to_period('M')  # e.g. 2024-11
monthly_revenue = (
    df.groupby('YearMonth')['Revenue']
    .sum()
    .reset_index()
    .rename(columns={'Revenue': 'MonthlyRevenue'})
)

# --- 5. Check how many full calendar months we have ------------------------------------
num_months = monthly_revenue['YearMonth'].nunique()
print(f"\nNumber of complete calendar months in data: {num_months}")

# --- 6. Output the monthly totals ------------------------------------------------------
print("\nMonthly Revenue Totals:")
print(monthly_revenue.to_string(index=False))

# --- 7. Seasonality statement ----------------------------------------------------------
if num_months < 24:
    print("\nSeasonality assessment: insufficient data (<24 months). Cannot reliably detect seasonality.")
else:
    print("\nSufficient data exists for seasonality analysis (>24 months).")

# --- 8. Optional: daily revenue within the 28-day window to visualise short trend ----
daily_revenue = (
    df.groupby(df['Order Date'].dt.date)['Revenue']
    .sum()
    .reset_index()
    .rename(columns={'Order Date': 'Date', 'Revenue': 'DailyRevenue'})
)

# Plot
plt.figure(figsize=(10, 4))
sns.lineplot(data=daily_revenue, x='Date', y='DailyRevenue')
plt.title("Daily Revenue Trend (28-day window)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('visualization/daily_revenue_trend.png', dpi=300)
plt.close()
```

## Solution Summary:

Monthly revenue summary (Jan-Dec 2024)

| Month | Revenue |
|-------|---------|
| Jan   | $792.00 |
| Feb   | $193.94 |
| Mar   | $1,338.70 |
| Apr   | $15.55 |
| May   | $55.50 |
| Jun   | $5,206.47 |
| Jul   | $71.37 |
| Aug   | $1,408.77 |
| Sep   | $5,715.74 |
| Oct   | $979.94 |
| Nov   | $2,569.98 |
| Dec   | $4,708.34 |

Key observations
- The data covers exactly 12 complete months (all of 2024).
- Revenue is highly uneven: two standout peaks in June (~$5.2k) and September (~$5.7k), with several low months (Feb, Apr, May, Jul) under $200.
- Because we only have one year of data, we cannot separate true seasonal patterns from one-off events; 24+ months are typically required to confirm seasonality.

**Chain Summary:**

| Metric | Value |
|---|---|
| Chain ID | 1769969201 |
| Prompt Tokens | 2834 |
| Completion Tokens | 1263 |
| Total Tokens | 4097 |
| Total Time (LLM Interact.) | 5.65 seconds |
| Average Response Speed | 223.38 tokens/second |
| Total Cost | $0.0000 |


## 🗄️ Mode 2: Natural Language SQL Agent

InSightAI can also work directly with SQL databases. Let's create a temporary database and query it using plain English.

In [ ]:
# Setup a demo SQLite database
conn = sqlite3.connect('demo_ecommerce.db')
cursor = conn.cursor()
cursor.execute('CREATE TABLE IF NOT EXISTS Orders (OrderID int, CustomerName text, TotalAmount real, OrderDate text)')
cursor.executemany('INSERT INTO Orders VALUES (?,?,?,?)', [
    (1, 'Alice Johnson', 250.0, '2023-01-15'),
    (2, 'Bob Smith', 150.0, '2023-01-20'),
    (3, 'Charlie Brown', 300.0, '2023-02-10'),
    (4, 'David Lee', 100.0, '2023-02-25'),
    (5, 'Eve Wilson', 500.0, '2023-03-05')
])
conn.commit()

# Point InSightAI to the database
ai.conn = conn

# Ask a SQL-related question
ai.pd_agent_converse("Who are my top 3 customers by revenue and how much have they spent?")

## 📄 Mode 3: Automated Report Generation

Finally, InSightAI has a high-level automated reporting mode. It will categorize your dataset, generate its own relevant questions, and produce a comprehensive summary.

In [ ]:
# Reset to CSV mode and enable report generation
ai.df = pd.read_csv('sales_data.csv')
delattr(ai, 'conn') # Remove DB connection to switch back to DF mode
ai.report_enabled = True
ai.report_question_count = 3

print("🚀 Starting Automated Report Generation...")
ai.pd_agent_converse(report) # This will trigger the report flow automatically